# Use Spark and batch deployment to predict customer churn with `ibm-watsonx-ai`

This notebook contains steps and code to develop a predictive model, and start scoring new data. This notebook introduces commands for getting data and for basic data cleaning and exploration, pipeline creation, model training, model persistance to Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.11 and Apache® Spark 3.4.

You will use a data set, **Telco Customer Churn**, which details anonymous customer data from a telecommunication company. Use the details of this data set to predict customer churn which is very critical to business as it's easier to retain existing customers rather than acquiring new ones.

## Learning goals

The learning goals of this notebook are:

-  Load a CSV file into an Apache® Spark DataFrame.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create an Apache® Spark machine learning pipeline.
-  Train and evaluate a model.
-  Persist a pipeline and model in Watson Machine Learning repository.
-  Explore and visualize prediction results using the plotly package.
-  Deploy a model for batch scoring using Watson Machine Learning API.


## Contents

This notebook contains the following parts:

1. [Set up the environment](#setup)
2. [Load and explore data](#load)
3. [Create spark ml model](#model)
4. [Persist model](#upload)
5. [Predict locally and visualize](#visualization)
6. [Deploy and score](#deploy)
7. [Clean up](#cleanup)
8. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Install and import the `ibm-watsonx-ai` and dependecies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install wget | tail -n 1
!pip install pyspark==3.4.3 | tail -n 1
!pip install -U ibm-watsonx-ai | tail -n 1

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `api_key`.

In [1]:
username = 'PASTE YOUR USERNAME HERE'
api_key = 'PASTE YOUR API_KEY HERE'
url = 'PASTE THE PLATFORM URL HERE'

In [ ]:
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=api_key,
    url=url,
    instance_id="openshift",
    version="5.1"
)

Alternatively you can use `username` and `password` to authenticate WML services.

```python
credentials = Credentials(
    username=***,
    password=***,
    url=***,
    instance_id="openshift",
    version="5.1"
)

```

**Note:** After installation/update of `ibm-watsonx-ai` package, please restart notebook kernel to make sure the installed version of the package is used.

In [2]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd5.1/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [3]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [4]:
client.set.default_space(space_id)

'SUCCESS'

### Test Spark

In [5]:
try:
    from pyspark.sql import SparkSession
except:
    print('Error: Spark runtime is missing. If you are using Watson Studio change the notebook runtime to Spark.')
    raise

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache® Spark DataFrame and perform a basic exploration. 

In [ ]:
import os
from wget import download

sample_dir = 'spark_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename = os.path.join(sample_dir, 'WA_FnUseC_TelcoCustomerChurn.csv')
if not os.path.isfile(filename):
    filename = download("https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd5.1/data/customer_churn/WA_FnUseC_TelcoCustomerChurn.csv", out=sample_dir)

In [ ]:
spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .option('nanValue', ' ')\
  .option('nullValue', ' ')\
  .load(filename)

Explore the loaded data by using the following Apache® Spark DataFrame methods:
-  print schema
-  count all records
-  show distribution of label classes

In [8]:
df_data.printSchema()

print("Number of fields: %3g" % len(df_data.schema))

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Churn: string (nullable = true)

Number of fields:  21


As you can see, the data contains 21 fields. "Churn" field is the one we would like to predict (label).

In [9]:
print("Total number of records: " + str(df_data.count()))

Total number of records: 7043


Data set contains 7043 records.

Now you will check if all records have complete data.

In [10]:
df_complete = df_data.dropna()

print("Number of records with complete data: %3g" % df_complete.count())

Number of records with complete data: 7032


You can see that there are some missing values you can investigate that all missing values are present in `TotalCharges` feature. We will use dataset with missing values removed for model training and evaluation.


Now you will inspect distribution of classes in label column.

In [11]:
df_complete.groupBy('Churn').count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|   No| 5163|
|  Yes| 1869|
+-----+-----+



<a id="model"></a>
## 3. Create an Apache® Spark machine learning model

In this section you will learn how to prepare data, create an Apache® Spark machine learning pipeline, and train a model.

### 3.1: Prepare data

In this subsection you will split your data into: train, test and predict datasets.

In [12]:
(train_data, test_data, predict_data) = df_complete.randomSplit([0.8, 0.18, 0.02], 24)

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))
print("Number of records for prediction: " + str(predict_data.count()))

Number of records for training: 5638
Number of records for evaluation: 1261
Number of records for prediction: 133


As you can see our data has been successfully split into three datasets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.
-  The predict data set will be used for prediction.

### 3.2: Create pipeline and train a model

In this section you will create an Apache® Spark machine learning pipeline and then train the model.

In the first step you need to import the Apache® Spark machine learning packages that will be needed in the subsequent steps.

In [14]:
from pyspark.ml.feature import StringIndexer, RFormula
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

In the following step, convert all the predictors to features vector and label feature convert to numeric.

In [15]:
lab = StringIndexer(inputCol = 'Churn', outputCol = 'label')
features = RFormula(formula = "~ gender + SeniorCitizen +  Partner + Dependents + tenure + PhoneService + MultipleLines + InternetService + OnlineSecurity + OnlineBackup + DeviceProtection + TechSupport + StreamingTV + StreamingMovies + Contract + PaperlessBilling + PaymentMethod + MonthlyCharges + TotalCharges - 1")

Next, define estimators you want to use for classification. Logistic Regression is used in the following example.

In [16]:
lr = LogisticRegression(maxIter = 10)

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [17]:
pipeline_lr = Pipeline(stages = [features, lab , lr])

Now, you can train your Logistic Regression model using the previously defined **pipeline** and **train data**."

In [ ]:
model_lr = pipeline_lr.fit(train_data)

You can check your **model accuracy** now. To evaluate the model, use **test data**.

In [19]:
predictions = model_lr.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Test dataset:")
print("Accuracy = %3.2f" % accuracy)

Test dataset:
Accuracy = 0.80


You can tune your model now to achieve better accuracy. For simplicity of this example tuning section is omitted.

<a id="upload"></a>
## 4. Persist model


In this section you will learn how to store your pipeline and model in Watson Machine Learning repository using Python client libraries.

**Note**: Apache® Spark 3.4 is required.

### 4.1: Save pipeline and model

In this subsection you will learn how to save pipeline and model artifacts to your Watson Machine Learning instance.

In [20]:
saved_model = client.repository.store_model(
    model=model_lr, 
    meta_props={
        client.repository.ModelMetaNames.NAME:'Customer Churn model',
        client.repository.ModelMetaNames.TYPE: "mllib_3.4",
        client.repository.ModelMetaNames.SOFTWARE_SPEC_ID: client.software_specifications.get_id_by_name('spark-mllib_3.4'),
        client.repository.ModelMetaNames.LABEL_FIELD: "Churn",
    },  
    training_data=train_data, 
    pipeline=pipeline_lr)

Get saved model metadata from Watson Machine Learning.

In [21]:
published_model_ID = client.repository.get_model_id(saved_model)

print("Model Id: " + str(published_model_ID))

Model Id: c35c567e-4704-4606-90b5-f07d1ffbee0e


**Model Id** can be used to retrive latest model version from Watson Machine Learning instance.

### 4.2: Load model

In this subsection you will learn how to load back saved model from specified instance of Watson Machine Learning.

In [22]:
loaded_model = client.repository.load(published_model_ID)

In [23]:
print(type(loaded_model))

<class 'pyspark.ml.pipeline.PipelineModel'>


As you can see the name is correct. You have already learned how save and load the model from Watson Machine Learning repository.

<a id="visualization"></a>
## 5. Predict locally

In this section you will learn how to load data from batch scoring and visualize the prediction results with plotly package.

### 5.1: Make local prediction using previously loaded model and test data

In this subsection you will score *predict_data* data set.

In [24]:
predictions = loaded_model.transform(predict_data)

Preview the results by calling the *show()* method on the predictions DataFrame.

In [25]:
predictions.show(5, truncate=False, vertical=True)

24/04/26 13:40:40 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------
 customerID       | 0117-LFRMW                                                                                                                    
 gender           | Male                                                                                                                          
 SeniorCitizen    | 0                                                                                                                             
 Partner          | Yes                                                                                                                           
 Dependents       | Yes                                                                                                                           
 tenure           | 37                                                                                                

By tabulating a count, you can see the split between labels.

In [26]:
predictions.select("prediction").groupBy("prediction").count().show(truncate=False)

+----------+-----+
|prediction|count|
+----------+-----+
|0.0       |103  |
|1.0       |30   |
+----------+-----+



<a id="deploy"></a>
## 6. Deploy model and score

In this section you will learn how to create batch deployment and to score a new data record by using the Watson Machine Learning REST API. 
For more information about REST APIs, see the [Swagger Documentation](https://cloud.ibm.com/apidocs/machine-learning#deployments-create).

### 6.1 Prepare scoring data for batch job

#### Get data for prediction

First, download scoring data into notebook's filesystem

In [ ]:
import os
from wget import download

sample_dir = 'spark_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename = os.path.join(sample_dir, 'scoreInput.csv')
if not os.path.isfile(filename):
    filename = download("https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd5.1/data/customer_churn/scoreInput.csv", out=sample_dir)

### 6.2: Create batch deployment

Now you can create a batch scoring endpoint.
Execute the following sample code that uses the **published_model_ID** value to create the scoring endpoint for predictions.

#### Create batch deployment for published model

In [28]:
meta_data = {
    client.deployments.ConfigurationMetaNames.NAME: "Customer Churn batch deployment",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {
        "name": "S",
        "num_nodes": 1
    }
}

deployment_details = client.deployments.create(published_model_ID, meta_props=meta_data)



######################################################################################

Synchronous deployment creation for id: 'c35c567e-4704-4606-90b5-f07d1ffbee0e' started

######################################################################################


ready.


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='d79c4305-978e-4eea-9112-d61997f70268'
-----------------------------------------------------------------------------------------------




Batch deployment has been created.

You can retrieve now your deployment ID.

In [29]:
deployment_id = client.deployments.get_id(deployment_details)

You can also list all deployments in your space.

In [ ]:
client.deployments.list()

If you want to get additional information on your deployment, you can do it as below.

In [30]:
client.deployments.get_details(deployment_id)

{'entity': {'asset': {'id': 'c35c567e-4704-4606-90b5-f07d1ffbee0e'},
  'batch': {},
  'custom': {},
  'deployed_asset_type': 'model',
  'hardware_spec': {'id': 'e7ed1d6c-2e89-42d7-aed5-863b972c1d2b',
   'name': 'S',
   'num_nodes': 1},
  'name': 'Customer Churn batch deployment',
  'space_id': 'cbd87244-b000-4279-b991-3cefbf8b1555',
  'status': {'state': 'ready'}},
 'metadata': {'created_at': '2024-04-26T11:41:36.998Z',
  'id': 'd79c4305-978e-4eea-9112-d61997f70268',
  'modified_at': '2024-04-26T11:41:36.998Z',
  'name': 'Customer Churn batch deployment',
  'owner': '1000330999',
  'space_id': 'cbd87244-b000-4279-b991-3cefbf8b1555'}}

<a id="score"></a>
#### Create and run Batch job

**Tip**: To install pandas execute `!pip install pandas`

In [31]:
import pandas as pd

score_input = pd.read_csv(filename).astype('object')

In [32]:
job_payload_ref = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [
        {
            "fields": score_input.columns.tolist(),
            "values": [score_input.loc[0].tolist()]
        }
    ]
}

In [33]:
job = client.deployments.create_job(deployment_id, meta_props=job_payload_ref)

Now, your job has been submitted to Spark runtime.

You can retrieve now your job ID.

In [34]:
job_id = client.deployments.get_job_id(job)

You can also list all jobs in your space.

In [ ]:
client.deployments.list_jobs()

If you want to get additional information on your job, you can do it as below.

In [36]:
client.deployments.get_job_details(job_id)

{'entity': {'deployment': {'id': 'd79c4305-978e-4eea-9112-d61997f70268'},
  'platform_job': {'job_id': '2a4631cd-b463-4221-b056-8bc0119afbce',
   'run_id': 'f580fd69-1277-4e19-a5d9-2d0bf9df44a4'},
  'scoring': {'input_data': [{'fields': ['customerID',
      'gender',
      'SeniorCitizen',
      'Partner',
      'Dependents',
      'tenure',
      'PhoneService',
      'MultipleLines',
      'InternetService',
      'OnlineSecurity',
      'OnlineBackup',
      'DeviceProtection',
      'TechSupport',
      'StreamingTV',
      'StreamingMovies',
      'Contract',
      'PaperlessBilling',
      'PaymentMethod',
      'MonthlyCharges',
      'TotalCharges',
      'Churn',
      'SampleWeight'],
     'values': [['9237-HQITU',
       'Female',
       0,
       'No',
       'No',
       2,
       'Yes',
       'No',
       'Fiber optic',
       'No',
       'No',
       'No',
       'No',
       'No',
       'No',
       'Month-to-month',
       'Yes',
       'Electronic check',
       70

#### Monitor job execution

Here you can check status of your batch scoring. When status of Spark job is `completed` the results will be written to scoring_output file in Object Storage.

In [37]:
import time

elapsed_time = 0
while client.deployments.get_job_status(job_id).get('state') != 'completed' and elapsed_time < 300:
    print(f" Current state: {client.deployments.get_job_status(job_id).get('state')}")
    elapsed_time += 10
    time.sleep(10)
if client.deployments.get_job_status(job_id).get('state') == 'completed':
    print(f" Current state: {client.deployments.get_job_status(job_id).get('state')}")
    job_details_do = client.deployments.get_job_details(job_id)
    print(job_details_do)
else:
    print("Job hasn't completed successfully in 5 minutes.")

 Current state: queued
 Current state: queued
 Current state: running
 Current state: completed
{'entity': {'deployment': {'id': 'd79c4305-978e-4eea-9112-d61997f70268'}, 'platform_job': {'job_id': '2a4631cd-b463-4221-b056-8bc0119afbce', 'run_id': 'f580fd69-1277-4e19-a5d9-2d0bf9df44a4'}, 'scoring': {'input_data': [{'fields': ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn', 'SampleWeight'], 'values': [['9237-HQITU', 'Female', 0, 'No', 'No', 2, 'Yes', 'No', 'Fiber optic', 'No', 'No', 'No', 'No', 'No', 'No', 'Month-to-month', 'Yes', 'Electronic check', 70.7, 151.65, 'Yes', 1]]}], 'predictions': [{'fields': ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'Inte

#### Get scored data

In [38]:
import json

print(json.dumps(client.deployments.get_job_details(job_id), indent=2))

{
  "entity": {
    "deployment": {
      "id": "d79c4305-978e-4eea-9112-d61997f70268"
    },
    "platform_job": {
      "job_id": "2a4631cd-b463-4221-b056-8bc0119afbce",
      "run_id": "f580fd69-1277-4e19-a5d9-2d0bf9df44a4"
    },
    "scoring": {
      "input_data": [
        {
          "fields": [
            "customerID",
            "gender",
            "SeniorCitizen",
            "Partner",
            "Dependents",
            "tenure",
            "PhoneService",
            "MultipleLines",
            "InternetService",
            "OnlineSecurity",
            "OnlineBackup",
            "DeviceProtection",
            "TechSupport",
            "StreamingTV",
            "StreamingMovies",
            "Contract",
            "PaperlessBilling",
            "PaymentMethod",
            "MonthlyCharges",
            "TotalCharges",
            "Churn",
            "SampleWeight"
          ],
          "values": [
            [
              "9237-HQITU",
              "F

<a id="cleanup"></a>
## 7. Clean up 

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd5.1/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 8. Summary and next steps     

You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. 
 
Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Amadeusz Masny**, Python Software Developer in Watson Machine Learning at IBM

Copyright © 2020-2025 IBM. This notebook and its source code are released under the terms of the MIT License.